__读取数据__

In [ ]:
import rasterio
from matplotlib import pyplot as plt
import numpy as np

# 打开ENVI格式的遥感数据文件
with rasterio.open("D:\ToolsRoming\py\sample.dat") as src:
    # 读取第2、3、4波段的数据
    band1 = src.read(1)  # 假设1是第二个波段
    band2 = src.read(2)  # 假设2是第三个波段
    band3 = src.read(3)  # 假设3是第四个波段

    # 将数据融合为RGB图像
    # 注意：这里需要确保所有波段的数据形状相同
    # 将第4波段作为红色，第3波段作为绿色通道，第2波段作为蓝色
    rgb_image = np.dstack((band3, band2, band1))

# 使用matplotlib展示RGB图像
plt.imshow(rgb_image)
plt.axis('off')  # 不显示坐标轴
plt.show()

__NDVI__

In [ ]:
import numpy as np
import rasterio
from rasterio.plot import show

# 打开ENVI格式的遥感数据文件
with rasterio.open("D:\ToolsRoming\py\sample.dat") as src:
    # 读取近红外和红色波段的数据
    # 确保这里的波段索引与你的数据匹配
    nir_band = src.read(4).astype(np.float16)  # 第四波段，近红外波段
    red_band = src.read(3).astype(np.float16)   # 第三波段，红色波段

    # 计算NDVI
    # 避免除以零，给分母加一个小的常数
    ndvi = (nir_band - red_band) / (nir_band + red_band)

# 显示NDVI图像
show(ndvi, cmap='gray')  # 使用适合NDVI的色图

__NDBI__

In [ ]:
import numpy as np
import rasterio
from rasterio.plot import show

# 打开ENVI格式的遥感数据文件
with rasterio.open("D:\ToolsRoming\py\sample.dat") as src:
    # 读取近红外和红色波段的数据
    # 确保这里的波段索引与你的数据匹配
    nir_band = src.read(4).astype(np.float16)   # 第四波段，近红外
    swir_band = src.read(5).astype(np.float16)  # 第五波段，短波

    # 计算NDVI
    # ndbi = (nir_band - swir_band) / (nir_band + swir_band)
    ndbi = (swir_band - nir_band) / (swir_band + nir_band)

# 显示NDVI图像
show(ndbi, cmap='gray')  # 使用适合NDVI的色图

__MNDWI__

In [ ]:
import numpy as np
import rasterio
from rasterio.plot import show

# 打开ENVI格式的遥感数据文件
with rasterio.open("D:\ToolsRoming\py\sample.dat") as src:
    # 读取近红外和红色波段的数据
    # 确保这里的波段索引与你的数据匹配
    green_band = src.read(2).astype(np.float16)  # 第五波段，短波
    swir_band = src.read(5).astype(np.float16)   # 第四波段，近红外

    # 计算NDVI
    # 避免除以零，给分母加一个小的常数
    mndwi = (green_band - swir_band) / (green_band + swir_band)

# 显示NDVI图像
show(mndwi, cmap='gray')  # 使用适合NDVI的色图

__显示图像的窗体__

In [ ]:
import tkinter as tk
from tkinter import messagebox
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import numpy as np
import rasterio

# 假设 mndwi 是一个已经计算好的 NumPy 数组
# mndwi = ...

def show_image_in_gui(image_data, title="Image"):
    # 创建一个 tkinter 窗体
    root = tk.Tk()
    root.title(title)

    # 创建一个 Matplotlib 图像
    fig = Figure(figsize=(10, 8))  # 可以根据需要调整图像大小
    ax = fig.add_subplot(111)
    cax = ax.imshow(image_data, cmap='gray', vmin=image_data.min(), vmax=image_data.max())
    fig.colorbar(cax)

    # 将 Matplotlib 图像嵌入到 tkinter 窗体中
    canvas = FigureCanvasTkAgg(fig, master=root)
    canvas.draw()
    canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)

    # 定义关闭窗体时的回调函数
    def on_closing():
        print('Window is closing...')
        root.destroy()

    # 绑定关闭事件
    root.protocol("WM_DELETE_WINDOW", on_closing)

    # 显示窗体
    root.mainloop()
